# retrieve run data from mongo

In [192]:
from pymongo import MongoClient
import polars as pl

client = MongoClient("localhost", 27017)
db = client["trader"]
documents = list(db["message"].find({}))
run_data = pl.DataFrame(documents)

filtered_data = run_data.filter(pl.col("trading_session_id") == run_data['trading_session_id'][-1])

preprocessed_data = [{
    'order_book': row['content'].get('order_book'),
    'incoming': {k: v for k, v in row['content'].get('incoming_message', {}).items() if k != 'trader_id'},
    'timestamp': row['timestamp']
} for row in filtered_data.to_dicts()]

new_order_books = pl.DataFrame(preprocessed_data).sort("timestamp", nulls_last=True).with_row_index("row_number")
print(new_order_books)

shape: (89, 4)
┌────────────┬───────────────────────────────────┬──────────────────┬─────────────────────────┐
│ row_number ┆ order_book                        ┆ incoming         ┆ timestamp               │
│ ---        ┆ ---                               ┆ ---              ┆ ---                     │
│ u32        ┆ struct[2]                         ┆ struct[3]        ┆ datetime[μs]            │
╞════════════╪═══════════════════════════════════╪══════════════════╪═════════════════════════╡
│ 0          ┆ {[],[]}                           ┆ {null,null,null} ┆ 2024-05-29 00:00:36.930 │
│ 1          ┆ {[],[{2006.0,1.0}]}               ┆ {1,2006.0,-1}    ┆ 2024-05-29 00:00:37.089 │
│ 2          ┆ {[{1994.0,1.0}],[{2006.0,1.0}]}   ┆ {1,1994.0,1}     ┆ 2024-05-29 00:00:37.613 │
│ 3          ┆ {[{1995.0,1.0}, {1994.0,1.0}],[{… ┆ {1,1995.0,1}     ┆ 2024-05-29 00:00:40.644 │
│ 4          ┆ {[{1996.0,1.0}, {1995.0,1.0}, {1… ┆ {1,1996.0,1}     ┆ 2024-05-29 00:00:41.505 │
│ …          ┆ …         

In [47]:
from analysis.utilities import delete_all_tables

delete_all_tables()

DuckDB tables deleted successfully.
MongoDB collection deleted successfully.
